In [1]:
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import MetricCollection, Accuracy, Specificity, Precision, F1Score

from datasets import SpectrogramDataset
from datasets.frames import CapgMyoDataModule, MyoArmbandDataModule, NinaProDataModule
from definitions import MODELS_FOLDER
from models import SequenceClassifier, CRNN, Classifier, OriginalModel
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.transforms import Compose, ToTensor, Normalize
import warnings
from functools import partial
from utils import lstm_cross_val_experiment

warnings.simplefilter(action='ignore', category=FutureWarning)

# Base Setup

In [2]:
max_epochs = 150
max_seq_epochs = 150
seed = 113
transform = Compose([
    ToTensor(),
    Normalize(0, 1)
])

callbacks = [partial(ModelCheckpoint, monitor='val/loss', dirpath=MODELS_FOLDER),
             partial(EarlyStopping, monitor='val/loss', patience=7)]

seq_callbacks = [partial(ModelCheckpoint, monitor='val/loss', dirpath=MODELS_FOLDER),
                 partial(EarlyStopping, monitor='val/loss', patience=17)]

sequence_model = partial(CRNN, num_layers=2, dropout=0.5, hidden_size=128)
partial_classifier = partial(Classifier, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001},
                             monitor='val/loss', sched_kwargs={'patience': 4},
                             time_window=[10, 20, 40], time_step=[1, 1, 1])
partial_seq_classifier = partial(SequenceClassifier, optim_kwargs={'lr': 0.001, 'weight_decay': 0.001},
                                 monitor='val/loss', sched_kwargs={'patience': 13})

# CapgMyo

In [3]:
data_module_capgmyo = CapgMyoDataModule(
    batch_size=10000,
    k_folds=10,
    train_transforms=transform,
    val_transforms=transform,
    test_transforms=transform,
    num_workers=32,
    seed=seed,
    shuffle_train=True,
    dataset=SpectrogramDataset,
    window_length=1
)

metrics = MetricCollection([Accuracy(average='micro', num_classes=data_module_capgmyo.num_classes),
                            Specificity(average='macro', num_classes=data_module_capgmyo.num_classes),
                            Precision(average='macro', num_classes=data_module_capgmyo.num_classes),
                            F1Score(average='macro', num_classes=data_module_capgmyo.num_classes)]).to(torch.device("cuda", 0))

## Window Length: 1

In [ ]:
lstm_cross_val_experiment(data_module=data_module_capgmyo, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'CapgMyo {data_module_capgmyo.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Window Length: 2

In [ ]:
data_module_capgmyo.set_window_length(2)

In [ ]:
lstm_cross_val_experiment(data_module=data_module_capgmyo, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'CapgMyo {data_module_capgmyo.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Window Length: 5

In [ ]:
data_module_capgmyo.set_window_length(5)

In [ ]:
lstm_cross_val_experiment(data_module=data_module_capgmyo, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'CapgMyo {data_module_capgmyo.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Window Length: 10

In [ ]:
data_module_capgmyo.set_window_length(10)

In [ ]:
lstm_cross_val_experiment(data_module=data_module_capgmyo, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'CapgMyo {data_module_capgmyo.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Clean

In [ ]:
data_module_capgmyo = None

# MyoArmband

In [3]:
data_module_myoarmband = MyoArmbandDataModule(
    batch_size=10000,
    k_folds=6,
    train_transforms=transform,
    val_transforms=transform,
    test_transforms=transform,
    num_workers=32,
    seed=seed,
    shuffle_train=True,
    dataset=SpectrogramDataset,
    window_length=1
)

metrics = MetricCollection([Accuracy(average='micro', num_classes=data_module_myoarmband.num_classes),
                            Specificity(average='macro', num_classes=data_module_myoarmband.num_classes),
                            Precision(average='macro', num_classes=data_module_myoarmband.num_classes),
                            F1Score(average='macro', num_classes=data_module_myoarmband.num_classes)]).to(torch.device("cuda", 0))

## Window Length: 1

In [4]:
lstm_cross_val_experiment(data_module=data_module_myoarmband, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'MyoArmband {data_module_myoarmband.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

Global seed set to 113
  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['metrics'])`.
  rank_zero_warn(
wandb: Cur

C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Science\EMG_Analysis\saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | OriginalModel    | 640 K 
1 | criterion | CrossEntropyLoss | 0     
2 | metrics   | MetricCollection | 0 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=2` reached.
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['metrics'])`.
  rank_zero_warn(

feature extraction: 10

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             Test metric                         DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
 test/10_1/majority_voting_Accuracy           0.7751666903495789
  test/10_1/majority_voting_F1Score           0.7801082134246826
 test/10_1/majority_voting_Precision          0.7918315529823303
test/10_1/majority_voting_Specificity          0.962526798248291
 test/20_1/majority_voting_Accuracy           0.7775312066078186
  test/20_1/majority_voting_F1Score           0.7824848294258118
 test/20_1/majority_voting_Precision          0.7946157455444336
test/20_1/majority_voting_Specificity         0.9629208445549011
 test/40_1/majority_voting_Accuracy           0.7805225849151611
  test/40_1/majority_voting_F1Score           0.7855209112167358
 test/40_1/majority_voting_Precision          

epoch,▁▁▁▅▅▅█
test/10_1/majority_voting_Accuracy,▁
test/10_1/majority_voting_F1Score,▁
test/10_1/majority_voting_Precision,▁
test/10_1/majority_voting_Specificity,▁
test/20_1/majority_voting_Accuracy,▁
test/20_1/majority_voting_F1Score,▁
test/20_1/majority_voting_Precision,▁
test/20_1/majority_voting_Specificity,▁
test/40_1/majority_voting_Accuracy,▁
test/40_1/majority_voting_F1Score,▁


100%|██████████| 3/3 [2:04:27<00:00, 2489.02s/it]


## Window Length: 2

In [ ]:
data_module_myoarmband.set_window_length(2)

In [ ]:
lstm_cross_val_experiment(data_module=data_module_myoarmband, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'MyoArmband {data_module_myoarmband.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Window Length: 5

In [5]:
data_module_myoarmband.set_window_length(5)

In [6]:
lstm_cross_val_experiment(data_module=data_module_myoarmband, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'MyoArmband {data_module_myoarmband.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

Global seed set to 113
  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['metrics'])`.
  rank_zero_warn(


C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Science\EMG_Analysis\saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | OriginalModel    | 1.7 M 
1 | criterion | CrossEntropyLoss | 0     
2 | metrics   | MetricCollection | 0 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=2` reached.
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['metrics'])`.
  rank_zero_warn(

feature extraction: 10

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             Test metric                         DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
 test/10_1/majority_voting_Accuracy           0.8413513898849487
  test/10_1/majority_voting_F1Score           0.8441604375839233
 test/10_1/majority_voting_Precision          0.8553344011306763
test/10_1/majority_voting_Specificity         0.9735580086708069
 test/20_1/majority_voting_Accuracy           0.8431415557861328
  test/20_1/majority_voting_F1Score           0.8460210561752319
 test/20_1/majority_voting_Precision          0.8574351072311401
test/20_1/majority_voting_Specificity         0.9738563299179077
 test/40_1/majority_voting_Accuracy           0.8453253507614136
  test/40_1/majority_voting_F1Score           0.8481611013412476
 test/40_1/majority_voting_Precision          

epoch,▁▁▁▅▅▅█
test/10_1/majority_voting_Accuracy,▁
test/10_1/majority_voting_F1Score,▁
test/10_1/majority_voting_Precision,▁
test/10_1/majority_voting_Specificity,▁
test/20_1/majority_voting_Accuracy,▁
test/20_1/majority_voting_F1Score,▁
test/20_1/majority_voting_Precision,▁
test/20_1/majority_voting_Specificity,▁
test/40_1/majority_voting_Accuracy,▁
test/40_1/majority_voting_F1Score,▁


100%|██████████| 3/3 [1:57:57<00:00, 2359.31s/it]


## Window Length: 10

In [ ]:
data_module_myoarmband.set_window_length(10)

In [ ]:
lstm_cross_val_experiment(data_module=data_module_myoarmband, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'MyoArmband {data_module_myoarmband.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Clean

In [7]:
data_module_myoarmband = None

# NinaPro

In [3]:
data_module_ninapro = NinaProDataModule(
    batch_size=5000,
    num_workers=32,
    k_folds=10,
    train_transforms=transform,
    val_transforms=transform,
    test_transforms=transform,
    seed=seed,
    shuffle_train=True,
    dataset=SpectrogramDataset,
    window_length=1
)

metrics = MetricCollection([Accuracy(average='micro', num_classes=data_module_ninapro.num_classes),
                            Specificity(average='macro', num_classes=data_module_ninapro.num_classes),
                            Precision(average='macro', num_classes=data_module_ninapro.num_classes),
                            F1Score(average='macro', num_classes=data_module_ninapro.num_classes)]).to(torch.device("cuda", 0))

## Window Length: 1

In [ ]:
lstm_cross_val_experiment(data_module=data_module_ninapro, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'NinaPro {data_module_ninapro.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

Global seed set to 113
  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['metrics'])`.
  rank_zero_warn(
wandb: Cur

C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Science\EMG_Analysis\saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | OriginalModel    | 711 K 
1 | criterion | CrossEntropyLoss | 0     
2 | metrics   | MetricCollection | 0 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\wandb\sdk\wandb_run.py", line 203, in check_status
    status_response = self._interface.communicate_stop_status()
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\wandb\sdk\interface\interface.py", line 127, in communicate_stop_status
    resp = self._communicate_stop_status(status)Exception in thread 
NetStatThr  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\wandb\sdk\interface\interface_sock.py", line 69, in _communicate_stop_status
:
Traceback (most recent call last):
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\threading.py", line 1009, in _bootstrap_inner
    data = super()._communicate_stop_s

## Window Length: 2

In [ ]:
data_module_ninapro.set_window_length(2)

In [ ]:
lstm_cross_val_experiment(data_module=data_module_ninapro, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'NinaPro {data_module_ninapro.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Window Length: 5

In [ ]:
data_module_ninapro.set_window_length(5)

In [ ]:
lstm_cross_val_experiment(data_module=data_module_ninapro, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'NinaPro {data_module_ninapro.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Window Length: 10

In [ ]:
data_module_ninapro.set_window_length(10)

In [ ]:
lstm_cross_val_experiment(data_module=data_module_ninapro, partial_classifier=partial_classifier,
                          partial_seq_classifier=partial_seq_classifier, name=f'NinaPro {data_module_ninapro.window_length}',
                          max_epochs=max_epochs, max_seq_epochs=max_seq_epochs, metrics=metrics, sequence_model=sequence_model,
                          callbacks=callbacks, seq_callbacks=seq_callbacks, seed=seed, model_checkpoint_index=0)

## Clean

In [ ]:
data_module_ninapro = None